## קוד מבוא

#### ספריות

In [1]:
import os
import sys
import pathlib
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### העלאת משתנים להרצת הקוד

In [3]:
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

future_folder = os.path.dirname(software_data_folder_location)

create_forecast_basic_software_location = os.path.dirname(future_folder)

sys.path.append(create_forecast_basic_software_location)

## פונקציות

### פונקציות גלובליות

In [4]:
from global_functions import up_load_shp, up_load_df, find_files_with_pattern

## code

### עיבוד חומר מקדים

##### העלאת טבלה

In [5]:
path=r'{}\background_files'.format(software_data_folder_location)

matching_files_age_dis_type=find_files_with_pattern(path, '_age_dis_type')
file_age_dis_type = pathlib.Path(matching_files_age_dis_type[0])
file_name_age_dis_type = file_age_dis_type.name[:-5]

taz_2020_catalog=up_load_df(path,file_name_age_dis_type)

##### העלאת טבלה

In [6]:

matching_files_2020_jtmt_forcast_full=find_files_with_pattern(r'{}'.format(forecast_version_folder_location), '2020_jtmt_forcast_full_')
file_2020_jtmt_forcast_full = pathlib.Path(matching_files_2020_jtmt_forcast_full[0])
file_name_2020_jtmt_forcast_full = file_2020_jtmt_forcast_full.name[:-5]

taz_2020_aprt=up_load_df(r'{}'.format(forecast_version_folder_location), file_name_2020_jtmt_forcast_full)

In [14]:
save_file=r'{}\Intermediates'.format(software_data_folder_location)

matching_files=find_files_with_pattern(save_file, '_kibolt_jew_till_2050')
file = pathlib.Path(matching_files[0])
file_name = file.name[:-5]

kibolt_jew_2050=up_load_df(save_file, file_name).set_index('Taz_num')

##### העלאת טבלה ושכבה

In [15]:
age_catalog=up_load_df(r'{}\background_files'.format(software_data_folder_location),'age_des_types')

taz_with_aprt_diversity=up_load_shp(r'{}\background_files\cost_diversity_for_age_distribution.shp'.format(software_data_folder_location)).set_index('Taz_num')

In [16]:

taz_2020_catalog=taz_2020_catalog.merge(taz_2020_aprt[['Taz_num','zonetype','aprt_20','pop_without_dorms_yeshiva','Taz_name','Muni_Heb']],on='Taz_num',how='left').set_index('Taz_num')

taz_2020_catalog['taz_with_aprt_diversity']=taz_with_aprt_diversity['div']

taz_2020_catalog['taz_expensive']=taz_with_aprt_diversity['cost']

col=['classification_name','age_cycle']

taz_2020_catalog=taz_2020_catalog.reset_index().merge(age_catalog[col],on='classification_name',how='left').set_index('Taz_num')

taz_2020_catalog=taz_2020_catalog.join(kibolt_jew_2050).query('sector_for_age != "Arab"')

taz_2020_catalog=taz_2020_catalog.fillna(0)

##### העלאת טבלה

In [17]:
folder_path=r'{}\background_files'.format(software_data_folder_location)

taz_sector_change=up_load_df(folder_path,'230704_taz_num_convert_from_jewish_to_u_orthodox')

In [18]:
lst_change_sector=list(taz_sector_change.Taz_num)

taz_2020_catalog.loc[(taz_2020_catalog.index.isin(lst_change_sector)),'sector_for_age']='U_Orthodox'

## חישוב 

In [19]:
singal_year=list(range(2025,2051,5))

In [20]:
years=['add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']

y_before=['2020','2020_2025',
 '2025_2030',
 '2030_2035',
 '2035_2040',
 '2040_2045']

only_years=['2020_2025',
 '2025_2030',
 '2030_2035',
 '2035_2040',
 '2040_2045',
 '2045_2050']

In [21]:
taz_2020_catalog['age_cycle_time']=0
taz_2020_catalog['current_aprt']=taz_2020_catalog['aprt_20']
years_to_change_cycle_stage=20
age_cycle_limit=4
taz_2020_catalog['age_cycle_2020']=taz_2020_catalog['age_cycle']
taz_2020_catalog['changed_this_cycle']=0

In [22]:
for y, y1 in zip(years, only_years):
    #### אזורים ללא יח"ד קיימות בשנת הבסיס מתאכלסים בהתפלגות גילים הראשוני
    taz_with_aprt_diversity_condition = taz_2020_catalog['taz_with_aprt_diversity']==1
    taz_expensive_condition = taz_2020_catalog['taz_expensive']==1
    
    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']==0)&
        (taz_2020_catalog['{}'.format(y)]>0)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,0,' | new diverse neighborhood | ',1]
    
    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']==0)&
        (taz_2020_catalog['{}'.format(y)]>0)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        taz_expensive_condition&
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2,0,' | new non-diverse expensive neighborhood  | ',1]
    
    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']==0)&
        (taz_2020_catalog['{}'.format(y)]>0)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        ~taz_expensive_condition&
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[1,0,' | new non-diverse neighborhood  | ',1]

    #### אזורים עם יח"ד קיימות בשנת הבסיס מתאכלסים בהתפלגות גילים כפונצקיה של יחס קיבולת למצב הקיים
    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']>0),'precent_add_to_current'
    ]=taz_2020_catalog['{}'.format(y)]/taz_2020_catalog['current_aprt']
    
    #### יחס נמוך - הקיבולת בטלה ביחס למצב הקיים
    #### יחס גדול - הקיבולת מבטלת את המצב הקיים
    kibolet_condition = taz_2020_catalog['precent_add_to_current'] > 0.5
    yush_condition =  taz_2020_catalog['zonetype']=='Judea and Samaria'

    taz_2020_catalog.loc[
        kibolet_condition &
        ~yush_condition &
        (taz_2020_catalog['changed_this_cycle']==0)&
        taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,0,'| renew diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        kibolet_condition &
        taz_expensive_condition &
        (taz_2020_catalog['changed_this_cycle']==0)&
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2,0,'| a lot of new aprts in non-diverse expensive neighborhood|',1]

    taz_2020_catalog.loc[
        kibolet_condition &
        (taz_2020_catalog['changed_this_cycle']==0)&
        ~taz_expensive_condition &
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[1,0,'| a lot of new aprts in non-diverse neighborhood|',1]

    #### יחס בינוני
    ##### ביוש אין התחדשות עירונית לכן אם יחס גבוהה יחסית זה כמו בנייה חדשה עם אכלוס של סוג התפלגות גילים הראשוני
    kibolet_condition =  taz_2020_catalog['precent_add_to_current']>= 0.20

    taz_2020_catalog.loc[
        (yush_condition)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        (kibolet_condition) , ['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ] = [1, 0,'| a lot of new aprts in non-diverse neighborhood in yush |',1]

    ##### יוש אין התחדשות עירונית לכן אם יחס לא גבוהה יחסית זה מונע התבגרות של המצב הקיים    
    ##### אם לא ביוש אז יש אפשרות התחדשות עירונית לכן המקום נהיה מגוון
    kibolet_condition =  (taz_2020_catalog['precent_add_to_current']>= 0.25) & (taz_2020_catalog['precent_add_to_current'] <= 0.5)

    taz_2020_catalog.loc[
        ~(yush_condition)&
        (kibolet_condition)&
        (taz_with_aprt_diversity_condition) , ['age_cycle','explanation_{}'.format(y1)]
    ] =  [3,'| a lot of new aprts in diverse neighborhood|']
    
    #השעון התקדם כבר ולא צריך לשנות כלום
    #### קוד לתהליך התבגרות התפלגות הגילים כפונקציה של סוג מגזר
    age_cycle_condition = taz_2020_catalog['age_cycle']<age_cycle_limit

    age_cycle_time_condition=(taz_2020_catalog['age_cycle_time']%years_to_change_cycle_stage==0)&(taz_2020_catalog['age_cycle_time']>0)

    sector_condition=taz_2020_catalog['sector_for_age']=='hitnachlut' 
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0)&
        (age_cycle_condition),['explanation_{}'.format(y1),'changed_this_cycle']
    ]=['| natural aging |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (age_cycle_condition),'age_cycle'
    ]=taz_2020_catalog['age_cycle']+1
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (age_cycle_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    sector_condition=taz_2020_catalog['sector_for_age']=='Jewish'

    age_cycle_condition = taz_2020_catalog['age_cycle']==1

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (taz_2020_catalog['changed_this_cycle']==0)&
        (sector_condition) &
        (age_cycle_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2 ,'| natural aging |',1]    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) & 
        (sector_condition) &
        (age_cycle_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    sector_condition=taz_2020_catalog['sector_for_age']=='Jewish'

    age_cycle_condition = taz_2020_catalog['age_cycle']==2
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| natural aging to a diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    sector_condition=taz_2020_catalog['sector_for_age']=='Jewish'

    age_cycle_condition = taz_2020_catalog['age_cycle']==4
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| old to a diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
    
    sector_condition=taz_2020_catalog['sector_for_age']=='U_Orthodox'

    age_cycle_condition = taz_2020_catalog['age_cycle']==1

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2 ,'| natural aging |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (age_cycle_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    age_cycle_condition = taz_2020_catalog['age_cycle']==2

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| natural aging to a diverse neighborhood |',1]
      
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
         (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition) &
        (sector_condition) &
        ~(taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[4,'| natural aging |',1]
    
    age_cycle_condition = taz_2020_catalog['age_cycle']==4
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| old to a diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
   
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        ~(taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    taz_2020_catalog.loc[(taz_2020_catalog['changed_this_cycle']==0),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
    
    taz_2020_catalog['changed_this_cycle']=0
    
    taz_2020_catalog['current_aprt']=taz_2020_catalog['current_aprt']+taz_2020_catalog['{}'.format(y)]
    taz_2020_catalog['current_aprt_{}'.format(y1)]= taz_2020_catalog['current_aprt']
    taz_2020_catalog['age_cycle_{}'.format(y1)]= taz_2020_catalog['age_cycle']
    taz_2020_catalog['age_cycle_time_{}'.format(y1)]= taz_2020_catalog['age_cycle_time']
    taz_2020_catalog['precent_add_to_current_{}'.format(y1)]= taz_2020_catalog['precent_add_to_current']

C:\Users\dpere\AppData\Local\Temp\ipykernel_4124\428659552.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value ' | new diverse neighborhood | ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  taz_2020_catalog.loc[
C:\Users\dpere\AppData\Local\Temp\ipykernel_4124\428659552.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value ' | new diverse neighborhood | ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  taz_2020_catalog.loc[
C:\Users\dpere\AppData\Local\Temp\ipykernel_4124\428659552.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value ' | new diverse neighborhood | ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  taz_2020_catalog.loc[
C:\Users\dpere\A

## ייצוא

In [23]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [24]:
taz_2020_catalog['hh_size_2020']=taz_2020_catalog['pop_without_dorms_yeshiva']/taz_2020_catalog['aprt_20']

In [25]:
for y1,sy,y_b, in zip(only_years, singal_year,y_before):
    taz_2020_catalog.loc[(taz_2020_catalog['age_cycle_{}'.format(y1)]==taz_2020_catalog['age_cycle_{}'.format(y_b)])&(taz_2020_catalog['age_cycle_{}'.format(y1)]==3),'hh_size_{}'.format(y1)]=taz_2020_catalog['hh_size_{}'.format(y_b)]
    taz_2020_catalog.loc[(taz_2020_catalog['age_cycle_{}'.format(y1)]==taz_2020_catalog['age_cycle_{}'.format(y_b)])&(taz_2020_catalog['age_cycle_{}'.format(y1)]==3),'hh_size_{}'.format(y1)]=taz_2020_catalog['hh_size_{}'.format(y1)]+0.15

In [26]:
col_age=[ 'pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

In [27]:
for y1,sy,y_b, in zip(only_years, singal_year,y_before):
    
    col_taz=['Taz_num','current_aprt_{}'.format(y1),'explanation_{}'.format(y1),'age_cycle_{}'.format(y1),'hh_size_{}'.format(y1),'sector_for_age']
    
    left_col=['sector_for_age','age_cycle_{}'.format(y1)]
    right_col=['sector','age_cycle']

    df=taz_2020_catalog.reset_index()[col_taz].merge(age_catalog,left_on=left_col,right_on=right_col,how='left').set_index('Taz_num')
    
    df['pop']=df['current_aprt_{}'.format(y1)]*df['hh_size_{}'.format(y1)]
    df.loc[df['hh_size_{}'.format(y1)].isna(),'pop']=df['current_aprt_{}'.format(y1)]*df['hh_size']
        
    df['pop_{}'.format(sy)]=df['pop']
    
    df['aprt_{}'.format(sy)]=df['current_aprt_{}'.format(y1)]
    
    df[col_age]=df[col_age].multiply(df['pop'], axis="index")    

    file_save=r'{}\Intermediates'.format(software_data_folder_location)

    save_excel_path=r'{}\{}_forecast_pop_jewish_{}_bau.xlsx'.format(file_save,file_date,y1)

    df.to_excel(save_excel_path,index=True)
    
    taz_2020_catalog=taz_2020_catalog.join(df[['pop_{}'.format(sy),'aprt_{}'.format(sy)]])